In [4]:
!wget -nc -O downloads/es_2014.zip "http://www.fega.es/sites/default/files/files/Beneficiaries_municipality_financial_year_2014.txt"

--2017-05-18 14:14:03--  http://www.fega.es/sites/default/files/files/Beneficiaries_municipality_financial_year_2014.txt
Resolving www.fega.es... 93.188.52.193
Connecting to www.fega.es|93.188.52.193|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 121303371 (116M) [text/plain]
Saving to: ‘downloads/es_2014.zip’

downloads/es_2014.z 100%[===================>] 115.68M  1.08MB/s    in 92s     

2017-05-18 14:15:35 (1.26 MB/s) - ‘downloads/es_2014.zip’ saved [121303371/121303371]



In [4]:
!wget -O downloads/es_2015.zip -nc "https://www.fega.es/sites/default/files/files/Beneficiaries_agricultural_district_financial_year_2015.zip"

--2017-05-17 20:51:14--  https://www.fega.es/sites/default/files/files/Beneficiaries_agricultural_district_financial_year_2015.zip
Resolving www.fega.es... 93.188.52.193
Connecting to www.fega.es|93.188.52.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19994764 (19M) [application/zip]
Saving to: ‘downloads/es_2015.zip’

downloads/es_2015.z 100%[===================>]  19.07M   363KB/s    in 59s     

2017-05-17 20:52:13 (332 KB/s) - ‘downloads/es_2015.zip’ saved [19994764/19994764]



In [5]:
!wget -O downloads/es_2015_muni.zip -nc "https://www.fega.es/sites/default/files/files/Beneficiaries_municipality_financial_ year_2015.zip"

--2017-05-17 20:52:13--  https://www.fega.es/sites/default/files/files/Beneficiaries_municipality_financial_%20year_2015.zip
Resolving www.fega.es... 93.188.52.193
Connecting to www.fega.es|93.188.52.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22769302 (22M) [application/zip]
Saving to: ‘downloads/es_2015_muni.zip’

downloads/es_2015_m 100%[===================>]  21.71M   322KB/s    in 66s     

2017-05-17 20:53:19 (337 KB/s) - ‘downloads/es_2015_muni.zip’ saved [22769302/22769302]



In [2]:
import pandas as pd

In [5]:
df_2015 = pd.read_csv('downloads/es_2015.zip', delimiter=';', encoding='latin1', compression='zip')
df_2015['year'] = 2015
df_2015 = df_2015.rename(columns={'COMARCA': 'MUNICIPIO'})
df_2015.head()

,BENEFICIARIO,PROVINCIA,MUNICIPIO,MEDIDA,IMPORTE_EUROS,year
0,A ALDEA,Ourense,3203 - VERIN,V/B.2.6 Non-productive investments [Med.216],"9914,25",2015
1,A BOUCIÑA S.C.,A Coruña,1502 - OCCIDENTAL,II.10 Reimbursement of appropriations carrie...,"462,58",2015
2,A BOUCIÑA S.C.,A Coruña,1502 - OCCIDENTAL,I.7 Specific support,"2069,54",2015
3,A BOUCIÑA S.C.,A Coruña,1502 - OCCIDENTAL,V/B.4.1 Implementing local development strat...,90000,2015
4,A BOUCIÑA S.C.,A Coruña,1502 - OCCIDENTAL,I.1 Single payment scheme,"17094,51",2015


In [6]:
df_2015_muni = pd.read_csv('downloads/es_2015_muni.zip', delimiter=';', encoding='latin1', compression='zip')
df_2015_muni['year'] = 2015
df_2015_muni = df_2015_muni.rename(columns={'COMARCA': 'MUNICIPIO'})
df_2015_muni.head()

,BENEFICIARIO,PROVINCIA,MUNICIPIO,MEDIDA,IMPORTE_EUROS,year
0,A ALDEA,Ourense,32030 - Entrimo,V/B.2.6 Non-productive investments [Med.216],"9914,25",2015
1,A BOUCIÑA S.C.,A Coruña,15074 - Rois,II.10 Reimbursement of appropriations carrie...,"462,58",2015
2,A BOUCIÑA S.C.,A Coruña,15074 - Rois,I.7 Specific support,"2069,54",2015
3,A BOUCIÑA S.C.,A Coruña,15074 - Rois,V/B.4.1 Implementing local development strat...,90000,2015
4,A BOUCIÑA S.C.,A Coruña,15074 - Rois,I.1 Single payment scheme,"17094,51",2015


In [7]:
len(df_2015), len(df_2015_muni)

(1575176, 1573255)

In [8]:
df_2014 = pd.read_csv('Beneficiaries_municipality_financial_year_2014.txt', delimiter=';', encoding='latin1')
df_2014['year'] = 2014
df_2014.head()

,BENEFICIARIO,PROVINCIA,MUNICIPIO,MEDIDA,IMPORTE_EUROS,year
0,A BOUCIÑA S.C.,A Coruña,15074 - Rois,I.1 Single payment scheme,"17105,53",2014
1,A BOUCIÑA S.C.,A Coruña,15074 - Rois,I.7 Specific support,"2830,77",2014
2,A CABADA SC,Lugo,27016 - Chantada,I.1 Single payment scheme,"9416,07",2014
3,A CABADA SC,Lugo,27016 - Chantada,I.7 Specific support,4241,2014
4,A CABADA SC,Lugo,27016 - Chantada,V/B.2.4 Agri-environment payments [Med.214],"9538,68",2014


In [9]:
df = pd.concat([df_2014, df_2015])
df.head()

,BENEFICIARIO,PROVINCIA,MUNICIPIO,MEDIDA,IMPORTE_EUROS,year
0,A BOUCIÑA S.C.,A Coruña,15074 - Rois,I.1 Single payment scheme,"17105,53",2014
1,A BOUCIÑA S.C.,A Coruña,15074 - Rois,I.7 Specific support,"2830,77",2014
2,A CABADA SC,Lugo,27016 - Chantada,I.1 Single payment scheme,"9416,07",2014
3,A CABADA SC,Lugo,27016 - Chantada,I.7 Specific support,4241,2014
4,A CABADA SC,Lugo,27016 - Chantada,V/B.2.4 Agri-environment payments [Med.214],"9538,68",2014


In [10]:
df['amount'] = pd.to_numeric(df['IMPORTE_EUROS'].str.replace(',', '.'))

In [11]:
df['recipient_postcode'] = df['MUNICIPIO'].str.split(' - ').apply(lambda x: x[0])
df.head()

,BENEFICIARIO,PROVINCIA,MUNICIPIO,MEDIDA,IMPORTE_EUROS,year,amount,recipient_postcode
0,A BOUCIÑA S.C.,A Coruña,15074 - Rois,I.1 Single payment scheme,"17105,53",2014,17105.53,15074
1,A BOUCIÑA S.C.,A Coruña,15074 - Rois,I.7 Specific support,"2830,77",2014,2830.77,15074
2,A CABADA SC,Lugo,27016 - Chantada,I.1 Single payment scheme,"9416,07",2014,9416.07,27016
3,A CABADA SC,Lugo,27016 - Chantada,I.7 Specific support,4241,2014,4241.00,27016
4,A CABADA SC,Lugo,27016 - Chantada,V/B.2.4 Agri-environment payments [Med.214],"9538,68",2014,9538.68,27016


In [12]:
df['recipient_location'] = df.apply(lambda x: '%s, %s' % (x['MUNICIPIO'].split(' - ')[1], x['PROVINCIA']), 1)
df.head()

,BENEFICIARIO,PROVINCIA,MUNICIPIO,MEDIDA,IMPORTE_EUROS,year,amount,recipient_postcode,recipient_location
0,A BOUCIÑA S.C.,A Coruña,15074 - Rois,I.1 Single payment scheme,"17105,53",2014,17105.53,15074,"Rois, A Coruña"
1,A BOUCIÑA S.C.,A Coruña,15074 - Rois,I.7 Specific support,"2830,77",2014,2830.77,15074,"Rois, A Coruña"
2,A CABADA SC,Lugo,27016 - Chantada,I.1 Single payment scheme,"9416,07",2014,9416.07,27016,"Chantada, Lugo"
3,A CABADA SC,Lugo,27016 - Chantada,I.7 Specific support,4241,2014,4241.00,27016,"Chantada, Lugo"
4,A CABADA SC,Lugo,27016 - Chantada,V/B.2.4 Agri-environment payments [Med.214],"9538,68",2014,9538.68,27016,"Chantada, Lugo"


In [13]:
from slugify import slugify

df = df.rename(columns={
    'BENEFICIARIO': 'recipient_name',
    'MEDIDA': 'scheme',
})
df = df.drop(['PROVINCIA', 'MUNICIPIO', 'IMPORTE_EUROS'], 1)
df['country'] = 'ES'
df['currency'] = 'EUR'
df['recipient_id'] = df.apply(lambda x: 'ES-%s-%s' % (x['recipient_postcode'], slugify(x['recipient_name'])), 1)
df.head()

,recipient_name,scheme,year,amount,recipient_postcode,recipient_location,country,currency,recipient_id
0,A BOUCIÑA S.C.,I.1 Single payment scheme,2014,17105.53,15074,"Rois, A Coruña",ES,EUR,ES-15074-A-BOUCINA-S-C
1,A BOUCIÑA S.C.,I.7 Specific support,2014,2830.77,15074,"Rois, A Coruña",ES,EUR,ES-15074-A-BOUCINA-S-C
2,A CABADA SC,I.1 Single payment scheme,2014,9416.07,27016,"Chantada, Lugo",ES,EUR,ES-27016-A-CABADA-SC
3,A CABADA SC,I.7 Specific support,2014,4241.00,27016,"Chantada, Lugo",ES,EUR,ES-27016-A-CABADA-SC
4,A CABADA SC,V/B.2.4 Agri-environment payments [Med.214],2014,9538.68,27016,"Chantada, Lugo",ES,EUR,ES-27016-A-CABADA-SC


In [14]:
df.to_csv('es_2014_2015.csv.gz', index=False, encoding='utf-8', compression='gzip')